In [1]:
import numpy as np
import pylab as plt
from scipy.stats.stats import pearsonr
import nltk
from nltk.tokenize import treebank
from nltk.corpus import opinion_lexicon
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestRegressor
import sys
from textblob import TextBlob

In [2]:
from collections import defaultdict
posW = opinion_lexicon.positive()
negW = opinion_lexicon.negative()
score = defaultdict(int)
for w in posW:
    score[w] = 1
for w in negW:
    score[w] = -1

In [3]:
markers = {"JJ" : 0,"JJR" : 0,"JJS" : 0,"NN" : 1,"NNP" : 1,"NNPS" : 1,"NNS" : 1,"PRP" : 1,"PRP$" : 1,"RB" : 2,
"RBR" : 2,"RBS" : 2,"VB" : 3,"VBD" : 3,"VBG" : 3,"VBN" : 3,"VBP" : 3,"VBZ" : 3} 

In [4]:
def get_sentiment(text):
    blob = TextBlob(text)
    sent=0.0
    for sentence in blob.sentences:
        sent+=sentence.sentiment.polarity
    sent /= len(blob.sentences)
    return (sent)

In [5]:
def get_ari():
    pass

In [6]:
def get_noun_count(tags):
    val = 0
    for t in tags:
        tag = t[1]
        if tag in markers.keys():
            if tag[0] == 'N':
                val += 1
    return val

In [7]:
def get_adj_count(review):
    val = 0
    for t in tags:
        tag = t[1]
        if tag in markers.keys():
            if tag[0] == 'J':
                val += 1
    return val

In [8]:
def get_verb_count(review):
    val = 0
    for t in tags:
        tag = t[1]
        if tag in markers.keys():
            if tag[0] == 'N':
                val += 1
    return val

In [67]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
from sklearn.decomposition import LatentDirichletAllocation

def do_LDA():
    file = open('reviews.json')
    reviews = []
    tokenizer = RegexpTokenizer(r'\w+')
    en_stop = stopwords.words('english')
    p_stemmer  = PorterStemmer()
    c = 0
    for l in file:
        line = eval(l)
        
        c = c+1
        if c == 3000:
            break
   
        try:
            tokens = tokenizer.tokenize(line['text'].lower())
            stopped_tokens = [i for i in tokens if i not in en_stop ]
            p_stemmer = PorterStemmer()
            tokens = [p_stemmer.stem(i) for i in stopped_tokens]
            reviews.append(tokens)
        except UnicodeDecodeError:
            print "F",
        
    dictionary = corpora.Dictionary(reviews)
    
    corpus = [dictionary.doc2bow(text) for text in reviews]
    ldamodel = models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=10)

    print ldamodel[corpus[0]]
    return ldamodel, dictionary

In [68]:
file = open('reviews.json')
lda, dictionary = do_LDA()

/home/sriram/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:23: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F F [(1, 0.082478507465740244), (2, 0.1271933693094891), (3, 0.25212461289234622), (4, 0.53133743722721571)]


#### Make datasets here

In [117]:
from nltk.tokenize import RegexpTokenizer
file = open('reviews.json')
tokenizer = RegexpTokenizer(r'\w+')
en_stop = stopwords.words('english')
p_stemmer  = PorterStemmer()
dataset = []
lenVsRating = {}

c = 0
for l in file:
    c = c+1
    line = eval(l)
    review = line['text']
    
    if c >= 1000:
        break
    
    try:
        tokens = nltk.word_tokenize(review.lower())
        text = nltk.Text(tokens)
        tags = nltk.pos_tag(text)
        
        tokens = tokenizer.tokenize(line['text'].lower())
        stopped_tokens = [i for i in tokens if i not in en_stop ]
        p_stemmer = PorterStemmer()
        tokens = [p_stemmer.stem(i) for i in stopped_tokens]
         
        
        bow = dictionary.doc2bow(tokens)
        
        # Features are nouns, length, sentiment, ratingz
        vec = [1.0]
        vec.append(get_verb_count(tags))
        vec.append(get_adj_count(tags))
        vec.append(len(review))
        vec.append(get_sentiment(review))
        
        ldaVec = [0]*10
        for val in lda[bow]:
            ldaVec[val[0]] = val[1]
        vec = vec + ldaVec
        
        vec.append(float(line['stars']))        
        dataset.append(vec) 
        
    except UnicodeDecodeError:
        continue

/home/sriram/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:24: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


In [118]:
dataset = np.array(dataset)
print dataset.shape

(919, 16)


In [119]:
X = dataset[:, :-1]
Y = dataset[:, -1]

idxs = np.random.choice(len(X), int(0.2*len(X)))
n_idx = [i for i in range(len(X)) if i not in idxs]

trainX = X[idxs]
trainY = Y[idxs]

testX  = X[n_idx]
testY = Y[n_idx]

In [120]:
print np.mean(trainY), np.mean(testY)

3.53551912568 3.42572944297


#### Train linear regression

In [123]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.svm import SVC

rgr = RandomForestRegressor(n_estimators=500, max_depth=4)
rgr.fit(trainX, trainY)

prediction = rgr.predict(testX)
print np.sum(np.abs(np.round(prediction)-testY))/len(testX)

0.840848806366


In [ ]:
nvec = np.array(dataset)

kys = lenVsRating.keys()
kys = sorted(kys)

x = []
y = []
for ky in kys:
    x.append(ky)
    y.append(np.round(np.mean(lenVsRating[ky])))


plt.scatter(np.array(x)*k, y, s = 150, color = 'navy', marker = '+')
plt.xlabel('Number of nouns used')
plt.ylabel('Rating')
print pearsonr(x, y)
plt.show()


# Len - -0.6262
# Verb - 0.561
# Adjective - 0.255
# Noun - -0.193
# Punctuation - 0.122
# Average word length - 0.035
# Average sentence length - 0.067
# Sentiment analysis - 0.61